# Depth Camera test

In [1]:
#! /usr/bin/env python3
import rospy
from ggcnn.srv import GraspPrediction
from sensor_msgs.msg import Image
import cv_bridge
bridge = cv_bridge.CvBridge()
import cv2
import helpers.transforms as tfh
from geometry_msgs.msg import PoseStamped, PoseArray
from std_msgs.msg import Header
from helpers.timeit import TimeIt
import numpy as np

In [ ]:
def process_depth_image(depth, crop_size=500, out_size=300, return_mask=False, crop_y_offset=20):
    imh, imw = depth.shape

    with TimeIt('1'):
        # Crop.
        depth_crop = depth[(imh - crop_size) // 2 - crop_y_offset:(imh - crop_size) // 2 + crop_size - crop_y_offset,
                           (imw - crop_size) // 2:(imw - crop_size) // 2 + crop_size]
    # depth_nan_mask = np.isnan(depth_crop).astype(np.uint8)

    # Inpaint
    # OpenCV inpainting does weird things at the border.
    with TimeIt('2'):
        depth_crop = cv2.copyMakeBorder(depth_crop, 1, 1, 1, 1, cv2.BORDER_DEFAULT)
        depth_nan_mask = np.isnan(depth_crop).astype(np.uint8)

    with TimeIt('3'):
        depth_crop[depth_nan_mask==1] = 0

    with TimeIt('4'):
        # Scale to keep as float, but has to be in bounds -1:1 to keep opencv happy.
        depth_scale = np.abs(depth_crop).max()
        depth_crop = depth_crop.astype(np.float32) / depth_scale  # Has to be float32, 64 not supported.

        with TimeIt('Inpainting'):
            depth_crop = cv2.inpaint(depth_crop, depth_nan_mask, 1, cv2.INPAINT_NS)

        # Back to original size and value range.
        depth_crop = depth_crop[1:-1, 1:-1]
        depth_crop = depth_crop * depth_scale

    with TimeIt('5'):
        # Resize
        depth_crop = cv2.resize(depth_crop, (out_size, out_size), cv2.INTER_AREA)

    if return_mask:
        with TimeIt('6'):
            depth_nan_mask = depth_nan_mask[1:-1, 1:-1]
            depth_nan_mask = cv2.resize(depth_nan_mask, (out_size, out_size), cv2.INTER_NEAREST)
        return depth_crop, depth_nan_mask
    else:
        return depth_crop

def depth_img_callback(msg):
    # Doing a rospy.wait_for_message is super slow, compared to just subscribing and keeping the newest one.
    curr_depth_img = bridge.imgmsg_to_cv2(msg)
    depth_crop, depth_nan_mask = process_depth_image(curr_depth_img,return_mask=True)
    cv2.imshow('Processed Depth Image', depth_crop)
    cv2.waitKey(1)
    
rospy.init_node("testjaja")

# Subscribe to the depth image topic
rospy.Subscriber("/camera/depth/image_rect_raw", Image, depth_img_callback)




# Actual service tests

In [2]:
#! /usr/bin/env python3
import rospy
from ggcnn.srv import GraspPrediction
from sensor_msgs.msg import Image
import cv_bridge
bridge = cv_bridge.CvBridge()
import cv2
import helpers.transforms as tfh
from geometry_msgs.msg import PoseStamped, PoseArray
from std_msgs.msg import Header
from helpers.timeit import TimeIt
import numpy as np


rospy.init_node("testjaja")
pose_pub = rospy.Publisher("/pose_viz", PoseArray, queue_size=1)
ggcnn_service_name = '/ggcnn_service'
rospy.wait_for_service(ggcnn_service_name + '/predict')
ggcnn_srv = rospy.ServiceProxy(ggcnn_service_name + '/predict', GraspPrediction)

In [5]:

# while True:
ret = ggcnn_srv.call()


best_grasp = ret.best_grasp


tfh.publish_pose_as_transform(best_grasp.pose, 'base_link', 'G', 0.5)
print(f'Your pose is: {best_grasp.pose}')


grasp_pose_stamped = PoseStamped(
    pose=best_grasp.pose, header=Header(frame_id="base_link")
)
pose_pub.publish(
    PoseArray(header=Header(frame_id="base_link"), poses=[best_grasp.pose])
)

Your pose is: position: 
  x: 31.492901863825296
  y: 134.06626385162474
  z: -604.1765036814106
orientation: 
  x: 0.9970603796069337
  y: 0.07661983697501132
  z: 4.691611905131982e-18
  w: 6.105234012211382e-17


In [52]:
import tensorflow as tf

# Path to your HDF5 file
model_path = '/home/riot/kinova_gen3_lite/src/ggcnn/scripts/models/epoch_29_model.hdf5'

# Load the model
model = tf.keras.models.load_model(model_path)

# Print the model summary to verify it loaded correctly
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 300, 300, 1)]        0         []                            
                                                                                                  
 conv2d_1 (Conv2D)           (None, 100, 100, 32)         2624      ['input_1[0][0]']             
                                                                                                  
 conv2d_2 (Conv2D)           (None, 50, 50, 16)           12816     ['conv2d_1[0][0]']            
                                                                                                  
 conv2d_3 (Conv2D)           (None, 25, 25, 8)            1160      ['conv2d_2[0][0]']            
                                                                                            